In [1]:
import random
from sys import platform
import clr
import numpy as np
import matplotlib.pyplot as plt
from jmetal.algorithm.singleobjective import GeneticAlgorithm
from jmetal.operator import SBXCrossover, PolynomialMutation, RandomSolutionSelection
from jmetal.util.termination_criterion import StoppingByKeyboard, StoppingByEvaluations
from jmetal.util.observer import PrintObjectivesObserver

from jmetal.core.quality_indicator import GenerationalDistance
from jmetal.core.problem import FloatProblem, FloatSolution, OnTheFlyFloatProblem
from jmetal.util.solution import (
    get_non_dominated_solutions,
    print_function_values_to_screen,
    print_variables_to_screen,
)

if platform == "win32":
    import pythoncom

    pythoncom.CoInitialize()
    PATH = "c:\\Users\\volkan.akkaya\\AppData\\Local\\DWSIM8\\"
elif platform == "linux":
    PATH = "/usr/local/lib/dwsim/"


clr.AddReference(PATH + "DWSIM.Automation.dll")
clr.AddReference(PATH + "DWSIM.Interfaces.dll")
clr.AddReference(PATH + "DWSIM.UnitOperations.dll")

from DWSIM.Automation import Automation2
from DWSIM.Interfaces.Enums import PropertyType

fs = Automation2()
sim = fs.LoadFlowsheet("orc6.dwxmz")

def print_properties(uo_name:str):
    global sim
    a = sim.GetFlowsheetSimulationObject(uo_name)
    for p in a.GetProperties(PropertyType.ALL):
        print(f"{p}: {a.GetPropertyValue(p)}")

In [9]:
splitter = sim.GetFlowsheetSimulationObject("SPLITTER")
orc10 = sim.GetFlowsheetSimulationObject("ORC10")
f = sim.GetFlowsheetSimulationObject("f")
g = sim.GetFlowsheetSimulationObject("g")
lpt = sim.GetFlowsheetSimulationObject("LP TURBINE")
wthp = sim.GetFlowsheetSimulationObject("Wthp")
wtlp = sim.GetFlowsheetSimulationObject("Wtlp")
wp = sim.GetFlowsheetSimulationObject("Wp")

sr = np.linspace(0.5, 1.0, 11)

for sri in sr:
    splitter.SetPropertyValue("SR1", float(sri))
    fs.CalculateFlowsheet2(sim)
    wtot = wtlp.GetPropertyValue("PROP_ES_0")+wthp.GetPropertyValue("PROP_ES_0")-wp.GetPropertyValue("PROP_ES_0")
    
    print(f"{sri}\t{f.GetPropertyValue('PROP_MS_27')}\t\
    {g.GetPropertyValue('PROP_MS_27')}\t\
    {orc10.GetPropertyValue('PROP_MS_27')}\t\
    {wtot}")

0.5	1.0	    1.0	    1.0	    10207.011928414902
0.55	1.0	    1.0	    1.0	    10303.814083360121
0.6	1.0	    1.0	    1.0	    10404.761259893752
0.65	1.0	    1.0	    1.0	    10505.290622939627
0.7	1.0	    1.0	    0.9893679475268833	    10606.167054282774
0.75	1.0	    1.0	    0.9738622516733317	    10706.94940028159
0.8	1.0	    1.0	    0.9604542320591756	    10807.535316732225
0.8500000000000001	0.953650138226716	    1.0	    0.8562480462158715	    10459.120997166337
0.9	0.8418142352743802	    0.9951195549308219	    0.6749574825466355	    9544.383191599904
0.95	0.7684667293138613	    0.9477047247282473	    0.5733313798078358	    8975.472880840374
1.0	0.7224402104931378	    0.9179512989488496	    0.5227213012023507	    8673.94722241403


In [16]:
lpt = sim.GetFlowsheetSimulationObject("LP TURBINE")
wthp = sim.GetFlowsheetSimulationObject("Wthp")
wtlp = sim.GetFlowsheetSimulationObject("Wtlp")
wp = sim.GetFlowsheetSimulationObject("Wp")
orc11a = sim.GetFlowsheetSimulationObject("ORC11a")
po = np.linspace(400e3,300e3,10)

import time
start = time.time()
for p in po:
    lpt.SetPropertyValue("PROP_TU_4", float(p))
    fs.CalculateFlowsheet2(sim)
    wtot = wtlp.GetPropertyValue("PROP_ES_0")+wthp.GetPropertyValue("PROP_ES_0")-wp.GetPropertyValue("PROP_ES_0")
    t = orc11a.GetPropertyValue("PROP_MS_0")
    print(f"{wtot} {t}")
stop = time.time()
print(stop-start)

10750.146971757222 304.9065363062054
10925.206498307985 314.9909238797987
11129.96464941826 313.9712282013117
11340.878973892168 312.92945434794905
11558.312729439025 311.8644359023193
11782.662002811821 310.7749066207643
12014.361322562783 309.65948855886194
12253.888086584031 308.5166784340933
12501.768474116958 307.3448319376167
12758.584416474325 306.1421456663137
47.42684030532837


In [10]:
pu = np.linspace(2.7e6, 3.5e6, 9)
feed_pump = sim.GetFlowsheetSimulationObject("FEED PUMP")
lpt = sim.GetFlowsheetSimulationObject("LP TURBINE")
wthp = sim.GetFlowsheetSimulationObject("Wthp")
wtlp = sim.GetFlowsheetSimulationObject("Wtlp")
wp = sim.GetFlowsheetSimulationObject("Wp")
orc10 = sim.GetFlowsheetSimulationObject("ORC10")
f = sim.GetFlowsheetSimulationObject("f")
for p in pu:
    feed_pump.SetPropertyValue("PROP_PU_5", float(p))
    fs.CalculateFlowsheet2(sim)
    wtot = wtlp.GetPropertyValue("PROP_ES_0")+wthp.GetPropertyValue("PROP_ES_0")-wp.GetPropertyValue("PROP_ES_0")
    x1 = orc10.GetPropertyValue("PROP_MS_27")
    x2 = f.GetPropertyValue("PROP_MS_27")
    print(f"{p} {wtot} {x1} {x2}")
    

2700000.0 10704.811575924417 0.9919977185673934 1.0
2800000.0 10730.661987487105 0.965600044568663 1.0
2900000.0 10741.031136126021 0.938192469881393 1.0
3000000.0 10732.815282268042 0.9080725204656841 0.9903245221353865
3100000.0 10705.370151259076 0.8746300614099156 0.9777124027402702
3200000.0 10651.612359497594 0.8352135020313677 0.9639671032586695
3300000.0 10566.854775125525 0.7872183511793561 0.948691592931316
3400000.0 10437.411306079612 0.7246180154811622 0.9307233905181087
3500000.0 10228.033347608443 0.6326814171541694 0.9071490697030348


In [6]:
class Problem(FloatProblem):
    def __init__(self):
        super(Problem, self).__init__()
    
        self.number_of_objectives = 1
        self.number_of_constraints = 3
        self.number_of_variables = 3
        self.lower_bound = [2500e3, 1000e3, 0.5]
        self.upper_bound = [3500e3, 2000e3, 1.0]
        
    def evaluate(self, solution: FloatSolution)->FloatSolution:
        # global fs
        # global sim
        # decision variables
        p_feed_pump = solution.variables[0]
        p_hp_turbine = solution.variables[1]
        # p_lp_turbine = solution.variables[2]
        sr1_splitter = solution.variables[2]
        
        # unit operations of decision variables
        feed_pump = sim.GetFlowsheetSimulationObject("FEED PUMP")
        hp_turbine = sim.GetFlowsheetSimulationObject("HP TURBINE")
        lp_turbine = sim.GetFlowsheetSimulationObject("LP TURBINE")
        splitter = sim.GetFlowsheetSimulationObject("SPLITTER")
        
        #unit operations of constraints
        orc10 = sim.GetFlowsheetSimulationObject("ORC10")
        f = sim.GetFlowsheetSimulationObject("f")
        g = sim.GetFlowsheetSimulationObject("g")
        orc11 = sim.GetFlowsheetSimulationObject("ORC11")
        
        # unit operations of objective function
        wthp = sim.GetFlowsheetSimulationObject("Wthp")
        wtlp = sim.GetFlowsheetSimulationObject("Wtlp")
        wp = sim.GetFlowsheetSimulationObject("Wp")
        
        feed_pump.SetPropertyValue("PROP_PU_5", p_feed_pump)
        hp_turbine.SetPropertyValue("PROP_TU_4", p_hp_turbine)
        # lp_turbine.SetPropertyValue("PROP_TU_4", p_lp_turbine)
        splitter.SetPropertyValue("SR1", sr1_splitter)
        
        fs.CalculateFlowsheet2(sim)
        
        
        wlp = wtlp.GetPropertyValue("PROP_ES_0")
        whp = wthp.GetPropertyValue("PROP_ES_0")
        wp = wp.GetPropertyValue("PROP_ES_0")
        
        xorc10 = orc10.GetPropertyValue("PROP_MS_27")
        xf = f.GetPropertyValue("PROP_MS_27")
        xg = g.GetPropertyValue("PROP_MS_27")
        # torc11 = orc11.GetPropertyValue("PROP_MS_0")
        
        solution.objectives= [-wlp-whp+wp]
        
        solution.constraints = [
            -xorc10+0.85,
            -xf+0.85,
            -xg+0.85,
           # -torc11+310,
           # p_hp_turbine-p_feed_pump,
           # p_lp_turbine-p_hp_turbine            
        ]
        
        # if sim.ErrorMessage!=None:
        #   print(sim.ErrorMessage[0:50])
        #    sim = fs.LoadFlowsheet("orc6.dxwmz")
        
        return solution
    
    def create_solution(self)->FloatSolution:
        new_solution = FloatSolution(
            number_of_objectives=self.number_of_objectives,
            number_of_constraints=self.number_of_constraints,
            lower_bound=self.lower_bound,
            upper_bound=self.upper_bound)
        
        new_solution.variables = [
            2500e3+(3500e3-2500e3)*random.random(),
            1000e3+(2000e3-1000e3)*random.random(),
            # 300e3+(400e3-300e3)*random.random(),
            0.5 + (1.0-0.5)*random.random()
        ]
        
        return new_solution
    
    def get_name(self)->str:
        return "ORC optimization"

In [8]:
problem = Problem()
algorithm = GeneticAlgorithm(
    problem=problem,
    population_size=50,
    offspring_population_size=50,
    mutation=PolynomialMutation(
        probability=1.0 / problem.number_of_variables, distribution_index=20
    ),
    crossover=SBXCrossover(probability=1.0, distribution_index=20),
    selection=RandomSolutionSelection(),
    termination_criterion=StoppingByEvaluations(max_evaluations=1000),
    #termination_criterion=StoppingByKeyboard(),
)
basic = PrintObjectivesObserver()
algorithm.observable.register(basic)

In [6]:
sol = algorithm.get_result()
print(sol.variables)
print(sol.constraints)
print(sol.objectives[0])

[2920047.630746697, 1191631.441430647, 300308.7740743387, 0.834006674646403]
[-0.06324221194217305, -0.15000000000000002, -0.15000000000000002, 5.058666980309681]
-12679.061399860531


In [7]:
algorithm.run()

2022-09-03 18:53:50,478 [MainThread  ] [INFO ]  Evaluations: 50. fitness: [-10590.439255890184]
2022-09-03 19:24:06,321 [MainThread  ] [INFO ]  Evaluations: 100. fitness: [-11966.418105050947]
2022-09-03 19:46:53,901 [MainThread  ] [INFO ]  Evaluations: 150. fitness: [-12346.466490739349]
2022-09-03 20:05:46,305 [MainThread  ] [INFO ]  Evaluations: 200. fitness: [-12346.466490739349]
2022-09-03 20:24:51,712 [MainThread  ] [INFO ]  Evaluations: 250. fitness: [-12420.754110948788]
2022-09-03 20:40:43,498 [MainThread  ] [INFO ]  Evaluations: 300. fitness: [-12538.620652854139]
2022-09-03 20:57:04,014 [MainThread  ] [INFO ]  Evaluations: 350. fitness: [-12538.620652854139]
2022-09-03 21:13:57,497 [MainThread  ] [INFO ]  Evaluations: 400. fitness: [-12538.620652854139]
2022-09-03 21:29:18,527 [MainThread  ] [INFO ]  Evaluations: 450. fitness: [-12608.94470107493]
2022-09-03 21:45:02,706 [MainThread  ] [INFO ]  Evaluations: 500. fitness: [-12610.173686731638]
2022-09-03 22:00:31,227 [MainThr

In [8]:
sol = algorithm.get_result()
print(sol.variables)
print(sol.constraints)
print(sol.objectives[0])


[3012781.3991107806, 1213392.2201064813, 300224.28437407245, 0.8384177420919912]
[-0.03283733981332404, -0.15000000000000002, -0.15000000000000002, 5.068185705530368]
-12690.912291216606


In [10]:
algorithm.start()

2022-09-04 01:23:14,113 [Thread-5    ] [INFO ]  Evaluations: 50. fitness: [-11524.129106545519]
2022-09-04 01:51:45,377 [Thread-5    ] [INFO ]  Evaluations: 100. fitness: [-12113.543788002104]
2022-09-04 02:17:25,226 [Thread-5    ] [INFO ]  Evaluations: 150. fitness: [-12113.543788002104]
2022-09-04 02:42:05,415 [Thread-5    ] [INFO ]  Evaluations: 200. fitness: [-12380.232007634499]
2022-09-04 03:05:05,635 [Thread-5    ] [INFO ]  Evaluations: 250. fitness: [-12380.232007634499]
2022-09-04 03:24:23,480 [Thread-5    ] [INFO ]  Evaluations: 300. fitness: [-12425.849008517083]
2022-09-04 03:43:52,591 [Thread-5    ] [INFO ]  Evaluations: 350. fitness: [-12520.221426558222]
2022-09-04 04:02:38,208 [Thread-5    ] [INFO ]  Evaluations: 400. fitness: [-12523.231969922363]
2022-09-04 04:22:31,525 [Thread-5    ] [INFO ]  Evaluations: 450. fitness: [-12523.231969922363]
2022-09-04 04:42:39,768 [Thread-5    ] [INFO ]  Evaluations: 500. fitness: [-12588.388060304396]
2022-09-04 05:02:46,044 [Thread

In [11]:
sol = algorithm.get_result()
print(sol.variables)
print(sol.constraints)
print(sol.objectives[0])

[2967918.8048006194, 1198053.4367124378, 300171.5113585641, 0.8340647066604355]
[-0.049119766279653865, -0.15000000000000002, -0.15000000000000002, 5.074132210774735]
-12683.349933360958


In [9]:
import time
start = time.time()
algorithm.run()
stop = time.time()
print(f"Elapsed time: {stop-start}")

2022-09-04 11:06:50,618 [MainThread  ] [INFO ]  Evaluations: 50. fitness: [-9560.688281418079]
2022-09-04 11:30:48,047 [MainThread  ] [INFO ]  Evaluations: 100. fitness: [-10839.643446726395]
2022-09-04 11:51:10,892 [MainThread  ] [INFO ]  Evaluations: 150. fitness: [-10869.967898477425]
2022-09-04 12:13:11,715 [MainThread  ] [INFO ]  Evaluations: 200. fitness: [-10893.998221426853]
2022-09-04 12:33:04,497 [MainThread  ] [INFO ]  Evaluations: 250. fitness: [-10893.998221426853]
2022-09-04 12:52:06,162 [MainThread  ] [INFO ]  Evaluations: 300. fitness: [-10893.998221426853]
2022-09-04 13:09:24,542 [MainThread  ] [INFO ]  Evaluations: 350. fitness: [-10902.78208044953]
2022-09-04 13:29:29,127 [MainThread  ] [INFO ]  Evaluations: 400. fitness: [-10902.78208044953]
2022-09-04 13:54:08,462 [MainThread  ] [INFO ]  Evaluations: 450. fitness: [-10909.581353941654]
2022-09-04 14:14:47,043 [MainThread  ] [INFO ]  Evaluations: 500. fitness: [-10909.581353941654]
2022-09-04 14:30:55,710 [MainThrea

Elapsed time: 24244.725164413452


In [12]:
res = algorithm.get_result()
print(res.variables)
print(res.constraints)
print(res.objectives[0])

[3046471.997268237, 1223162.806700811, 0.8379382441184651]
[-0.022880598055368906, -0.15000000000000002, -0.15000000000000002]
-10920.574123880644


In [13]:
import time
start = time.time()
algorithm.run()
stop = time.time()
print(f"Elapsed time: {stop-start}")

2022-09-04 23:19:09,111 [MainThread  ] [INFO ]  Evaluations: 50. fitness: [-10148.041512644904]
2022-09-04 23:51:24,195 [MainThread  ] [INFO ]  Evaluations: 100. fitness: [-10803.979630954113]
2022-09-05 00:16:44,564 [MainThread  ] [INFO ]  Evaluations: 150. fitness: [-10803.979630954113]
2022-09-05 00:40:17,227 [MainThread  ] [INFO ]  Evaluations: 200. fitness: [-10803.979630954113]
2022-09-05 01:04:58,233 [MainThread  ] [INFO ]  Evaluations: 250. fitness: [-10827.069668816412]
2022-09-05 01:29:15,484 [MainThread  ] [INFO ]  Evaluations: 300. fitness: [-10827.069668816412]
2022-09-05 01:49:07,026 [MainThread  ] [INFO ]  Evaluations: 350. fitness: [-10902.302942285283]
2022-09-05 02:10:03,327 [MainThread  ] [INFO ]  Evaluations: 400. fitness: [-10902.302942285283]
2022-09-05 02:30:03,644 [MainThread  ] [INFO ]  Evaluations: 450. fitness: [-10902.302942285283]
2022-09-05 02:48:34,593 [MainThread  ] [INFO ]  Evaluations: 500. fitness: [-10903.078644231246]
2022-09-05 03:06:27,362 [MainTh

Elapsed time: 26083.468015432358


In [14]:
res = algorithm.get_result()
print(res.variables)
print(res.constraints)
print(res.objectives[0])

[3071910.034346933, 1229729.5602135935, 0.8385514158825744]
[-0.013842318921038044, -0.15000000000000002, -0.15000000000000002]
-10918.375109424307
